In [1]:
import pdfplumber
import pandas as pd
import sys
import os
import contextlib

# pdf_path = r"C:\Users\dpere\Documents\python-projects\web-gis-scraper-desktop-electron\backend\101-0925198_תדפיס הוראות התכנית_חתום לאישור_1.pdf"
pdf_path = r"C:\Users\dpere\Documents\python-projects\web-gis-scraper-desktop-electron\backend\101-0135004_תדפיס הוראות התכנית_חתום לאישור_1.pdf"

search_phrase = "טבלת זכויות והוראות בניה"[::-1]

In [2]:
@contextlib.contextmanager
def suppress_stderr():
    with open(os.devnull, "w") as devnull:
        old_stderr = sys.stderr
        sys.stderr = devnull
        try:
            yield
        finally:
            sys.stderr = old_stderr

In [3]:
def find_table_start_page(pdf_path: str, phrase: str) -> int | None:
    with suppress_stderr():
        with pdfplumber.open(pdf_path) as pdf:
            for i, page in enumerate(pdf.pages):
                text = page.extract_text()
                if text and phrase in text:
                    return i
        return None


start_page = find_table_start_page(pdf_path, search_phrase)

In [4]:
def extract_raw_table_from_pdf(pdf_path: str, start_page: int) -> pd.DataFrame:
    """
    מחלץ טבלה גולמית מ־PDF החל מעמוד מסוים.
    ממשיך עמוד אחר עמוד עד שאין טבלאות.
    לא מניח מראש את שמות העמודות – מחזיר DataFrame גולמי.
    """
    all_rows = []

    with suppress_stderr():
        with pdfplumber.open(pdf_path) as pdf:
            page_index = start_page
            while page_index < len(pdf.pages):
                page = pdf.pages[page_index]
                tables = page.extract_tables()

                if not tables:
                    break

                return tables

In [5]:
# table1 = extract_raw_table_from_pdf(pdf_path, start_page)
# # table2 = extract_raw_table_from_pdf(pdf_path, start_page + 1)
# # table3 = extract_raw_table_from_pdf(pdf_path, start_page + 2)

# # row1 = table1[1][3]
# # row2 = table1[0][0]
# # row2 = table1[1][1]
# # row3 = table1[1][2]
# # row4 = table[1][3]
# # print((row1))
# # print(len(row1))


def find_table_end_page(pdf_path: str, start_page: int) -> int:
    with suppress_stderr():
        with pdfplumber.open(pdf_path) as pdf:
            for i in range(start_page, len(pdf.pages)):
                page = pdf.pages[i]
                tables = page.extract_tables()

                if not tables:
                    return i - 1

                for table in tables:
                    if not table or not table[0] or not table[0][0]:
                        continue

                    first_cell = table[0][0]
                    clean_cell = first_cell.replace("\n", "").replace(" ", "").strip()

                    # זיהוי התחלת טבלת הוראות לפי מילים ברורות
                    if "עצומבצמ" in clean_cell and "תוארוה" in clean_cell:
                        return i - 1

            return len(pdf.pages) - 1


# find_table_end_page(pdf_path, start_page)
start_page = find_table_start_page(pdf_path, search_phrase)
end_page = find_table_end_page(pdf_path, start_page)

print(f"הטבלה מתחילה בעמוד {start_page} ונגמרת בעמוד {end_page}")

הטבלה מתחילה בעמוד 21 ונגמרת בעמוד 20


In [11]:
import pdfplumber


def are_headers_similar(header_row1, header_row2, threshold=0.8):
    """בודק דמיון בין שורות כותרת."""
    if len(header_row1) != len(header_row2):
        return False
    similar_count = 0
    for i in range(len(header_row1)):
        if (
            header_row1[i]
            and header_row2[i]
            and SequenceMatcher(
                None, header_row1[i].strip(), header_row2[i].strip()
            ).ratio()
            > threshold
        ):
            similar_count += 1
    return similar_count / len(header_row1) > threshold


def identify_quantitative_table(pdf_path, start_page):
    quantitative_table = []
    found_start = False
    with pdfplumber.open(pdf_path) as pdf:
        for i in range(len(pdf.pages)):
            page_num = i + 1
            page = pdf.pages[i]
            tables = page.extract_tables()

            if page_num >= start_page:
                if not found_start and tables:
                    # נניח שהטבלה הראשונה בעמוד ההתחלה היא תחילת הטבלה הכמותית
                    quantitative_table.extend(tables[0])
                    initial_headers = tables[0][0] if tables[0] else None
                    found_start = True
                    continue

                if found_start and tables:
                    for table in tables:
                        if not quantitative_table:  # מקרה קצה של טבלה בעמוד אחד
                            quantitative_table.extend(table)
                            return quantitative_table

                        current_headers = table[0] if table else None

                        # בדיקת המשכיות כותרות (אם קיימות כותרות בטבלה הנוכחית)
                        headers_match = False
                        if (
                            initial_headers
                            and current_headers
                            and len(initial_headers) == len(current_headers)
                        ):
                            if are_headers_similar(initial_headers, current_headers):
                                headers_match = True

                        # בדיקת סוגי נתונים (דוגמה פשוטה - האם רוב התאים מכילים מספרים?)
                        is_mostly_numeric = False
                        if table and len(table) > 1:
                            numeric_count = 0
                            total_cells = 0
                            for row in table[1:]:
                                for cell in row:
                                    if (
                                        cell
                                        and cell.strip().replace(".", "", 1).isdigit()
                                    ):
                                        numeric_count += 1
                                    total_cells += 1
                            if total_cells > 0 and numeric_count / total_cells > 0.6:
                                is_mostly_numeric = True

                        # קריטריונים נוספים (מבנה, מילות מפתח שליליות וכו') יכולים להתווסף כאן

                        if headers_match or is_mostly_numeric:  # שילוב קריטריונים
                            quantitative_table.extend(
                                table[1:]
                            )  # הוספת שורות אחרי הכותרת
                        else:
                            # אם הטבלה לא נראית כהמשך, סביר להניח שסיימנו
                            return quantitative_table
        return quantitative_table


from difflib import SequenceMatcher

# דוגמת שימוש (יש להחליף את הנתיב ואת מספר העמוד)
pdf_file = r"C:\Users\dpere\Documents\python-projects\web-gis-scraper-desktop-electron\backend\101-0135004_תדפיס הוראות התכנית_חתום לאישור_1.pdf"

start_page_number = 22
result_table = identify_quantitative_table(pdf_file, start_page_number)

if result_table:
    print("הטבלה הכמותית שנמצאה:")
    for row in result_table:
        print(row)
else:
    print("לא נמצאה טבלה כמותית רציפה החל מהעמוד שצוין.")

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, def

הטבלה הכמותית שנמצאה:
['עצומ בצמ - הינב תוארוהו תויוכז תלבט', '.5']
